In [16]:
import sqlite3
import random
import pandas as pd
import numpy as np
from faker import Faker
from dataclasses import dataclass

# Configuração do Faker para dados brasileiros
fake = Faker('pt_BR')

@dataclass
class Imovel:
    """Classe de dados representando a entidade Imóvel."""
    area: float
    quartos: int
    idade_imovel: int
    bairro: str
    preco: float = 0.0  # Será calculado baseados nos outros atributos

class GerenciadorDB:
    """Responsável por todas as interações com o SQLite."""
    
    def __init__(self, db_name="imoveis_ml.db"):
        self.db_name = db_name
        self.conn = None
        self._conectar()
        self._criar_tabela()

    def _conectar(self):
        self.conn = sqlite3.connect(self.db_name)

    def _criar_tabela(self):
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS imoveis (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                area REAL,
                quartos INTEGER,
                idade_imovel INTEGER,
                bairro TEXT,
                preco REAL
            )
        ''')
        self.conn.commit()

    def salvar_lote(self, lista_imoveis):
        """Insere uma lista de objetos Imovel no banco."""
        cursor = self.conn.cursor()
        dados = [
            (i.area, i.quartos, i.idade_imovel, i.bairro, i.preco) 
            for i in lista_imoveis
        ]
        cursor.executemany('''
            INSERT INTO imoveis (area, quartos, idade_imovel, bairro, preco)
            VALUES (?, ?, ?, ?, ?)
        ''', dados)
        self.conn.commit()
        print(f"{len(lista_imoveis)} imóveis salvos no banco de dados.")

    def carregar_dados_dataframe(self):
        """Lê os dados do banco e retorna um DataFrame do Pandas."""
        return pd.read_sql_query("SELECT * FROM imoveis", self.conn)

    def fechar(self):
        if self.conn:
            self.conn.close()

In [17]:
class GeradorDeDados:
    """Fabrica dados fictícios com padrões matemáticos."""
    
    def __init__(self):
        self.bairros = ['Centro', 'Jardins', 'Botafogo', 'Vila Nova', 'Lagoa']
    
    def _calcular_preco_ficticio(self, area, quartos, idade):
        # Preço base por metro quadrado + valor por quarto - depreciação por idade
        preco_base = (area * 3500) + (quartos * 15000) - (idade * 500)
        
        # Adiciona um "ruído" aleatório de +/- 10% para não ficar perfeito demais
        ruido = random.uniform(0.9, 1.1)
        return round(max(preco_base * ruido, 50000), 2) # Garante preço mínimo

    def gerar_imoveis(self, quantidade=100):
        lista_imoveis = []
        for _ in range(quantidade):
            area = round(random.uniform(40, 250), 2)
            quartos = random.randint(1, 5)
            idade = random.randint(0, 50)
            bairro = random.choice(self.bairros)
            
            preco = self._calcular_preco_ficticio(area, quartos, idade)
            
            novo_imovel = Imovel(area, quartos, idade, bairro, preco)
            lista_imoveis.append(novo_imovel)
            
        return lista_imoveis

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

class EstimadorPreco:
    """Classe que encapsula o modelo de Machine Learning."""
    
    def __init__(self):
        self.modelo = LinearRegression()
        self.X_test = None
        self.y_test = None
        self.y_pred = None

    def treinar(self, df):
        print("\nTreinando modelo de Machine Learning...")
        
        # Selecionando Features (X) e Target (y)
        # Note: Em um caso real, converteríamos 'bairro' (texto) para números (encoding)
        # Por simplicidade, vamos usar apenas as variáveis numéricas aqui.
        X = df[['area', 'quartos', 'idade_imovel']]
        y = df['preco']

        # Divisão Treino/Teste
        X_train, self.X_test, y_train, self.y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        self.modelo.fit(X_train, y_train)
        print("Modelo treinado com sucesso.")

    def avaliar(self):
        if self.X_test is None:
            print("Erro: Treine o modelo antes de avaliar.")
            return

        self.y_pred = self.modelo.predict(self.X_test)
        mae = mean_absolute_error(self.y_test, self.y_pred)
        r2 = r2_score(self.y_test, self.y_pred)

        print("\n--- Resultados da Avaliação ---")
        print(f"R² (Precisão do ajuste): {r2:.4f} (O ideal é próximo de 1.0)")
        print(f"Erro Médio Absoluto: R$ {mae:,.2f}")
        
    def prever_novo_imovel(self, area, quartos, idade):
        entrada = pd.DataFrame([[area, quartos, idade]], 
                               columns=['area', 'quartos', 'idade_imovel'])
        predicao = self.modelo.predict(entrada)[0]
        return predicao

In [19]:
if __name__ == "__main__":
    # 1. Inicializar Banco de Dados
    db = GerenciadorDB()

    # 2. Gerar Dados Fictícios
    print("Gerando dados sintéticos...")
    gerador = GeradorDeDados()
    dados_novos = gerador.gerar_imoveis(quantidade=1000) # Gerar 1000 imóveis
    
    # 3. Salvar no SQLite
    db.salvar_lote(dados_novos)

    # 4. Carregar dados para o Pandas
    df_imoveis = db.carregar_dados_dataframe()
    print(f"\nAmostra dos dados:\n{df_imoveis.head()}")

    # 5. Treinar Modelo de ML
    ml_system = EstimadorPreco()
    ml_system.treinar(df_imoveis)
    ml_system.avaliar()

    # 6. Teste Manual
    print("\nTeste de Previsão:")
    area_teste = 100
    quartos_teste = 3
    idade_teste = 5
    
    valor_previsto = ml_system.prever_novo_imovel(area_teste, quartos_teste, idade_teste)
    print(f"Imóvel de {area_teste}m², {quartos_teste} quartos, {idade_teste} anos.")
    print(f"Valor Estimado pelo IA: R$ {valor_previsto:,.2f}")

    # Limpeza
    db.fechar()

Gerando dados sintéticos...
1000 imóveis salvos no banco de dados.

Amostra dos dados:
   id    area  quartos  idade_imovel    bairro      preco
0   1  226.86        1             6     Lagoa  785806.66
1   2  158.36        2            16   Jardins  552910.79
2   3  123.35        5            11    Centro  462908.65
3   4   63.52        2            15   Jardins  251680.70
4   5   59.00        2            32  Botafogo  200837.20

Treinando modelo de Machine Learning...
Modelo treinado com sucesso.

--- Resultados da Avaliação ---
R² (Precisão do ajuste): 0.9738 (O ideal é próximo de 1.0)
Erro Médio Absoluto: R$ 27,702.05

Teste de Previsão:
Imóvel de 100m², 3 quartos, 5 anos.
Valor Estimado pelo IA: R$ 391,699.21
